In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.model_selection import cross_val_score

## Evaluation metric

In [2]:
def f1_weighted(y_true, y_pred):
    score = f1_score(y_true, y_pred,average="weighted")
    return score
from sklearn.metrics import f1_score, make_scorer
scorer = make_scorer(f1_weighted, greater_is_better=True)

## Data

In [3]:
train= pd.read_csv("https://raw.githubusercontent.com/dphi-official/Datasets/master/sukhna_dhanas/train_set_label.csv" )
test = pd.read_csv('https://raw.githubusercontent.com/dphi-official/Datasets/master/sukhna_dhanas/test_set_label.csv')

In [5]:
train.head()

,Solidity,Eccentricity,EquivDiameter,Extrema,FilledArea,Extent,Orientation,EulerNumber,BoundingBox1,BoundingBox2,BoundingBox3,BoundingBox4,ConvexHull1,ConvexHull2,ConvexHull3,ConvexHull4,MajorAxisLength,MinorAxisLength,Perimeter,ConvexArea,Centroid1,Centroid2,Area,microorganism
0,0.711636,0.673498,0.109069,0.870544,0.010808,0.660599,0.094353,0.987915,0.870476,0.447276,0.029022,0.058923,0.875395,0.875395,0.877460,0.871746,0.025669,0.028256,0.010776,0.001776,0.878873,0.453973,0.020195,1
1,0.184271,0.865533,0.181675,0.306442,0.021235,0.183804,0.484926,0.974488,0.284444,0.345343,0.148896,0.115320,0.304870,0.304870,0.295238,0.290794,0.128062,0.077815,0.044747,0.016309,0.325508,0.360384,0.045702,2
2,0.431175,0.679469,0.172644,0.750469,0.020929,0.380940,0.891717,0.946626,0.707302,0.227592,0.066246,0.109428,0.728653,0.728653,0.729524,0.723810,0.065495,0.062696,0.049242,0.006400,0.739531,0.255860,0.042004,2
3,0.712849,0.991839,0.240241,0.271420,0.036976,0.700643,0.016835,0.975159,0.268571,0.468366,0.023344,0.249158,0.270715,0.270715,0.269841,0.268571,0.146561,0.020286,0.035455,0.006678,0.267614,0.568813,0.073303,4
4,0.338077,0.996782,0.123578,0.045654,0.011389,0.088682,0.219150,0.982544,0.041905,0.630931,0.100946,0.187710,0.041746,0.041746,0.041905,0.041905,0.134704,0.011038,0.034491,0.004833,0.071762,0.699979,0.024521,2


In [6]:
train.dtypes

Solidity           float64
Eccentricity       float64
EquivDiameter      float64
Extrema            float64
FilledArea         float64
Extent             float64
Orientation        float64
EulerNumber        float64
BoundingBox1       float64
BoundingBox2       float64
BoundingBox3       float64
BoundingBox4       float64
ConvexHull1        float64
ConvexHull2        float64
ConvexHull3        float64
ConvexHull4        float64
MajorAxisLength    float64
MinorAxisLength    float64
Perimeter          float64
ConvexArea         float64
Centroid1          float64
Centroid2          float64
Area               float64
microorganism        int64
dtype: object

In [7]:
train.shape , test.shape

((13824, 24), (3456, 23))

In [8]:
train.corr()

,Solidity,Eccentricity,EquivDiameter,Extrema,FilledArea,Extent,Orientation,EulerNumber,BoundingBox1,BoundingBox2,BoundingBox3,BoundingBox4,ConvexHull1,ConvexHull2,ConvexHull3,ConvexHull4,MajorAxisLength,MinorAxisLength,Perimeter,ConvexArea,Centroid1,Centroid2,Area,microorganism
Solidity,1.000000,-0.269357,-0.402124,0.197959,-0.299701,0.903390,-0.185676,0.400483,0.330461,0.037502,-0.584136,-0.569033,0.302313,0.302313,0.321032,0.318844,-0.587595,-0.458858,-0.559122,-0.319480,0.225622,-0.108761,-0.351460,0.157682
Eccentricity,-0.269357,1.000000,0.101508,-0.110107,0.005057,-0.431907,0.051134,-0.024092,-0.151711,-0.008445,0.184855,0.208251,-0.144460,-0.144460,-0.147555,-0.141932,0.296730,-0.109796,0.027703,-0.015578,-0.119439,0.041852,0.024020,0.206781
EquivDiameter,-0.402124,0.101508,1.000000,0.070673,0.839844,-0.388498,0.129568,-0.849679,-0.101808,-0.171998,0.750545,0.796699,-0.063728,-0.063728,-0.091433,-0.089198,0.721353,0.713264,0.853001,0.487830,0.039563,0.029204,0.926452,0.337338
Extrema,0.197959,-0.110107,0.070673,1.000000,0.047499,0.194285,-0.011359,-0.092980,0.962775,-0.018372,0.042456,-0.017340,0.966757,0.966757,0.963573,0.965019,-0.007023,0.006206,0.002367,-0.013910,0.982043,-0.017374,0.048442,0.117208
FilledArea,-0.299701,0.005057,0.839844,0.047499,1.000000,-0.255434,0.050477,-0.918808,-0.081690,-0.169650,0.598326,0.679517,-0.054807,-0.054807,-0.075577,-0.069130,0.591877,0.624446,0.794992,0.495957,0.034042,-0.004495,0.957957,0.116700
Extent,0.903390,-0.431907,-0.388498,0.194285,-0.255434,1.000000,-0.195469,0.338348,0.327530,0.013393,-0.579165,-0.537263,0.300550,0.300550,0.317567,0.314001,-0.575101,-0.358818,-0.471911,-0.260424,0.223685,-0.124783,-0.312079,0.031439
Orientation,-0.185676,0.051134,0.129568,-0.011359,0.050477,-0.195469,1.000000,-0.068410,-0.113199,-0.174051,0.151979,0.083955,-0.080773,-0.080773,-0.105306,-0.112952,0.096807,0.083641,0.114990,-0.001087,-0.089323,-0.151211,0.079849,0.037507
EulerNumber,0.400483,-0.024092,-0.849679,-0.092980,-0.918808,0.338348,-0.068410,1.000000,0.048024,0.176280,-0.668223,-0.766036,0.012604,0.012604,0.038298,0.034071,-0.682777,-0.701883,-0.863655,-0.575899,-0.081799,-0.010756,-0.939738,-0.072191
BoundingBox1,0.330461,-0.151711,-0.101808,0.962775,-0.081690,0.327530,-0.113199,0.048024,1.000000,0.015670,-0.159529,-0.154442,0.994231,0.994231,0.998499,0.998359,-0.169853,-0.117356,-0.167254,-0.100962,0.982349,-0.020560,-0.104459,0.072693
BoundingBox2,0.037502,-0.008445,-0.171998,-0.018372,-0.169650,0.013393,-0.174051,0.176280,0.015670,1.000000,-0.125880,-0.147739,0.002117,0.002117,0.012658,0.014697,-0.133682,-0.176676,-0.160221,-0.151689,-0.008066,0.967889,-0.183802,0.018973


In [10]:
train.describe()

,Solidity,Eccentricity,EquivDiameter,Extrema,FilledArea,Extent,Orientation,EulerNumber,BoundingBox1,BoundingBox2,BoundingBox3,BoundingBox4,ConvexHull1,ConvexHull2,ConvexHull3,ConvexHull4,MajorAxisLength,MinorAxisLength,Perimeter,ConvexArea,Centroid1,Centroid2,Area
count,13824.000000,13824.000000,13824.000000,13824.000000,13824.000000,13824.000000,13824.000000,13824.000000,13824.000000,13824.000000,13824.000000,13824.000000,13824.000000,13824.000000,13824.000000,13824.000000,13824.000000,13824.000000,13824.000000,13824.000000,13824.000000,13824.000000,13824.000000
mean,0.483543,0.816668,0.158836,0.528733,0.027930,0.390554,0.480102,0.967930,0.483085,0.444948,0.085813,0.123932,0.494234,0.494234,0.487924,0.487700,0.082797,0.043939,0.049383,0.012236,0.515745,0.479954,0.048216
std,0.209131,0.168313,0.111070,0.238477,0.048621,0.218697,0.310809,0.045840,0.239790,0.227774,0.088206,0.111279,0.236573,0.236573,0.238699,0.239473,0.086908,0.052404,0.068806,0.041491,0.239394,0.224588,0.065958
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000073,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.298202,0.719613,0.091416,0.352069,0.008160,0.213847,0.165487,0.962403,0.307823,0.266257,0.028391,0.052066,0.320683,0.320683,0.317745,0.311429,0.025956,0.020286,0.011265,0.001776,0.336157,0.314578,0.015655
50%,0.466642,0.869592,0.135782,0.524431,0.015266,0.354430,0.522795,0.980866,0.459683,0.449033,0.063432,0.090909,0.478052,0.478052,0.463492,0.462857,0.057924,0.028227,0.026138,0.003972,0.508826,0.462994,0.028598
75%,0.669901,0.938896,0.209592,0.715447,0.031614,0.523621,0.752532,0.990260,0.661587,0.590510,0.114676,0.162458,0.676154,0.676154,0.668571,0.673651,0.112127,0.056314,0.057580,0.010660,0.710310,0.658128,0.058514
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000




*   There are no missing values in the data
*   Data is already scaled [0,1]
*   Dataset is well balanced (each class is equally distributed)













In [9]:
target=train.pop("microorganism")

## Feature Engineering

In [11]:
train["1"]=train.Eccentricity*train.Orientation

In [12]:
train["2"]=train.Eccentricity*train.BoundingBox2

In [13]:
train["3"]=train.Area*train.Extent

Some other featues can be created which may increase the score

## MODELING 

### Model-1

In [15]:
from sklearn.ensemble import ExtraTreesClassifier
etc=ExtraTreesClassifier()
scores = cross_val_score(etc,train,target, cv = 5, scoring = scorer)
(scores.mean())

0.9963802745488615

### Model-2

In [16]:
from lightgbm import LGBMClassifier
lgb=LGBMClassifier()
scores = cross_val_score(lgb, train,target, cv = 5, scoring = scorer)
scores.mean()

0.9892730617593886

### Model-3

In [17]:
from xgboost import XGBClassifier
xgb=XGBClassifier()
scores = cross_val_score(xgb, train,target, cv = 5, scoring = scorer)
scores.mean()

0.9197340551232124

### Final Model

In [18]:
from sklearn.ensemble import StackingClassifier
estimator=[("xgb",xgb),("lgb",lgb),("etc",etc)]
clf = StackingClassifier(estimators=estimator)

In [ ]:
scores = cross_val_score(clf, train,target, cv = 5, scoring = scorer)
scores.mean()

0.997177301888153

## Predictions

In [ ]:
test["1"]=test.Eccentricity*test.Orientation
test["2"]=test.Eccentricity*test.BoundingBox2
test["3"]=test.Area*test.Extent

In [ ]:
clf.fit(data,value)
pred=clf.predict(test)

In [ ]:
res = pd.DataFrame(pred) 
res.index = test.index
res.columns = ["prediction"]
res.to_csv("sub.csv",index=False)

## Things that where not tried-

In [ ]:
!pip install -U pandas-profiling
from pandas_profiling import ProfileReport
profile=ProfileReport(train,title="Report",explorative=True)

In [ ]:
#This loads the report of the dataset describing each feature.
#Unable to load it as the notebook wasn't uploading.
profile.to_notebook_iframe()

### On Further analysis of data-

*   There was huge duplicate data in the train & test set.
*   Non of the features are normally distributed.
*   There were highly correlated features.
*   More feature engineering could be done on the data.



